# Text classification

The task of automatically categorizing texts into some classes. 

Popular methods of text classification: Naive Bayes, Logistic Regression, Support Vector Machines. 

### Naive Bayes Classifier. Наивный Байесовский классификатор. 

We call it naive cause we assume that all features are independent

In [1]:
import re

In [2]:
def extract_features(text, features):
    lst = re.findall('\w+', text.lower())
    v = []
    dct = {}
    for w in lst:
        if w not in dct:
            dct[w] = 1
        else:
            dct[w] += 1
    
    for i in features:
        #v.append(lst.count(i))    #сount очень дорогая операция,  т.е здесь сложность = квадрат
        if i in dct:
            v.append(dct[i])
        else:
            v.append(0)
    return v


In [3]:
text = (open('input.txt', encoding='UTF-8')).read()
features = sorted(open('features.txt', encoding='UTF-8').read().split())
print(extract_features(text, features))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]


In [4]:
def train_nbc(features, corpora, classes):
    if classes[0] != 'positive':
        classes[0], classes[1] =  classes[1], classes[0]
    lst=[[], []]
    
    for i,e in enumerate(corpora):
        tokens = re.findall('\w+', e.lower())
        dct = {}
        d = [t for t in tokens if t in features]
        for w in d:
            if w not in dct:
                dct[w] = 1
            else:
                dct[w] += 1
    
        V = len(d)
        #print(len(features))
        #V = len(tokens)    #все слова, НО надо считать только кол-во фичей
        V_extra = len(set(d))    #это нужно, чтобы частоты ФИЧЕЙ V_pos/V_neg были сопоставимы
        for j in features:
            if j in dct:
                lst[i].append((dct[j] + 1)/(V + V_extra))
                #lst[0].append((dct[j])/(V))
            else:
                lst[i].append(0)
                
    return lst

In [5]:
def classify(text, features, classes, priors, Params):
    p1 = 1.0
    p2 = 1.0    
    
    v = extract_features(text, features)
    for i in range(len(features)):
        k = v[i]
        
        #if Params[0][i] != 0.0:
        #    p1 *= (Params[0][i] ** k)
        #if Params[1][i] != 0.0:
        #    p2 *= (Params[1][i] ** k)
        p1 *= (Params[0][i] ** k)
        p2 *= (Params[1][i] ** k)
    positive_probability = priors[0] * p1
    negative_probability = priors[1] * p2
    
    print('positive_probability', positive_probability)
    print('negative_probability', negative_probability)
    
    if positive_probability == negative_probability:
        return 'equal'
    elif positive_probability > negative_probability:
        return 'positive'
    else:
        return 'negative'   

In [6]:
features = sorted(open('features.txt', encoding='UTF-8').read().split())

corpus_pos = open('pos_train.txt', encoding='UTF-8').read()
corpus_neg = open('neg_train.txt', encoding='UTF-8').read()
corpora = corpus_pos, corpus_neg    #tuple

classes = 'positive', 'negative'

Params = train_nbc(features, corpora, classes)

priors = 0.5, 0.5
text = (open('input.txt', encoding='UTF-8')).read()

label = classify(text, features, classes, priors, Params)
print(label)

positive_probability 8.235317441580478e-12
negative_probability 4.171376137406484e-11
negative


#feat = [...]    #O(n) сложность операции
#feat = {...}    #O(1)